In [ ]:
# import pandas as pd
# import plotly.express as px
# import plotly.graph_objects as go
# import streamlit as st

# data = pd.read_csv("C:/Users/Johnny/OneDrive/Desktop/382 FILES/ArabBarometer_WaveVIII_English_v3.csv")

# data.head()

,COUNTRY,WT,DATE,STRATUM,PSU,BLOCK_ID,ID,Q1A_PAL,Q1,Q13,...,Q1025,Q116,Q1014A,Q1014B,Q1015,Q1015A,Q1016,Q1010C,Q1010D_1,Q1010D_2
0,7,0.869931,2024-05-13,70003,174,NaN,700001,NaN,70003,2.0,...,2,2,1,97,250000,NaN,2,NaN,2.0,1.0
1,7,1.159908,2024-05-13,70003,174,NaN,700002,NaN,70003,2.0,...,2,1,4,4,600000,NaN,3,NaN,1.0,1.0
2,7,0.263755,2024-05-15,70001,274,NaN,700003,NaN,70001,1.0,...,2,2,1,6,800000,NaN,2,NaN,NaN,NaN
3,7,3.048913,2024-05-15,70001,274,NaN,700004,NaN,70001,1.0,...,2,2,1,0,500000,NaN,2,NaN,1.0,1.0
4,7,1.175850,2024-05-15,70014,59,NaN,700005,NaN,70014,1.0,...,2,2,1,1,600000,NaN,2,NaN,1.0,1.0


In [1]:
# app.py  – quick test for GCC-obesity dataset
import streamlit as st
import pandas as pd

# ---------- 1.  LOAD DATA ---------------------------------
@st.cache_data  # speeds up reloads
def load_data():
    return pd.read_csv("C:/Users/Johnny/OneDrive/Desktop/382 FILES/BEFA58B_ALL_LATEST.csv")

df = load_data()

# keep only rows with an obesity rate and the four GCC countries
gcc_countries = [
    "Bahrain", "Kuwait", "Oman", "Qatar",
    "Saudi Arabia", "United Arab Emirates"
]
df = df[df["GEO_NAME_SHORT"].isin(gcc_countries)]

# ---------- 2.  SIDEBAR FILTERS ---------------------------
st.sidebar.header("Choose a view")
country = st.sidebar.selectbox(
    "Country", sorted(df["GEO_NAME_SHORT"].unique())
)

gender_map = {
    "Both": "TOTAL",
    "Male": "MALE",
    "Female": "FEMALE"
}
gender_label = st.sidebar.radio(
    "Gender", list(gender_map.keys()), horizontal=True
)
gender_code = gender_map[gender_label]

# ---------- 3.  FILTERED SUBSET ---------------------------
sub = df[
    (df["GEO_NAME_SHORT"] == country) &
    (df["DIM_SEX"] == gender_code)
]

if sub.empty:
    st.warning(
        "No data for this gender in the country selected – "
        "try another option."
    )
    st.stop()

# pick the *latest* year with an estimate
latest_year = sub["DIM_TIME"].max()
latest_row  = sub[sub["DIM_TIME"] == latest_year].iloc[0]
rate        = latest_row["RATE_PER_100_N"]  # prevalence %

# ---------- 4.  DISPLAY -----------------------------------
st.title("GCC Obesity Dashboard – quick test")

st.metric(
    f"Obesity rate in {country} ({gender_label}) – {latest_year}",
    f"{rate:.1f} %"
)

# optional: simple trend line
st.line_chart(
    sub.sort_values("DIM_TIME").set_index("DIM_TIME")["RATE_PER_100_N"],
    height=250
)

st.caption(
    "Source: BEFA58B_ALL_LATEST.csv – RATE_PER_100_N = prevalence "
    "of adults (18+) with BMI ≥ 30 kg/m²."
)


KeyboardInterrupt: 